Convert PDFs to text

In [ ]:
# %pip install PyPDF2
# %pip install openpyxl==3.0.10
# %pip install pdfplumber
# %pip install tika
# %pip install PyMuPDF pandas
# %pip install pdf2image pytesseract pillow
# %pip install nltk
# %pip install spacy



'%%python' is not recognized as an internal or external command,
operable program or batch file.


In [38]:
#Core
import os
import json
import pandas as pd
import re #regular expression


#OCR
from pdf2image import convert_from_path
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
import fitz  # PyMuPDF

# Tokenizers
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import spacy
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 3500000

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lwert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
def ocr_on_pdfs(pdf_dir):
    #identify the dataset
    pdf_data = []

    # Loop through all PDF files in the directory
    for filename in os.listdir(pdf_dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_dir, filename)
            print("working on:", file_path)

            try:

                #convert file to image
                images = convert_from_path(file_path, poppler_path=r"C:\poppler-24.08.0\Library\bin")
                # Extract the text from all pages
                text = []
                for img in images:
                    text.append(pytesseract.image_to_string(img))

                    # Get PDF metadata
                    # Open the PDF file
                doc = fitz.open(file_path)
                metadata = doc.metadata
                page_count = doc.page_count


                # Create a dictionary to convert to dataframe
                pdf_data.append({
                    'filename': filename,
                    'text': text,
                    'page_count': page_count,
                    'creator': metadata.get('creator'),
                    'producer': metadata.get('producer'),
                    'subject': metadata.get('subject'),
                    'keywords': metadata.get('keywords'),
                    'title': metadata.get('title'),
                    'creation_date': metadata.get('creationDate'),
                })

                doc.close()

            except Exception as e:
                print(f"OCR error on {filename}: {e}")

    return pdf_data



In [7]:
#pull pdf data and run through ocr
pdf_dir = "pdf_data"
pdfs = ocr_on_pdfs(pdf_dir)

# Save to JSON
with open('legal_pdf_data.json', 'w', encoding='utf-8') as f:
    json.dump(pdfs, f, ensure_ascii=False, indent=2)

# Convert returned dictionary to DataFrame and save
df = pd.DataFrame(pdfs)
# df.to_csv('legal_pdfs_dataset.csv', index=False)
df.to_json('legal_pdf_data.json', orient='records', force_ascii=False, indent=2)


working on: pdf_data\ARG-1472-D-2023.pdf
working on: pdf_data\CA-2885.pdf
working on: pdf_data\CA-AB2013.pdf
working on: pdf_data\CA-AJR6.pdf
working on: pdf_data\CA-SB1047.pdf
working on: pdf_data\CA-SB942.pdf
working on: pdf_data\CO-CAIA.pdf
working on: pdf_data\DE-H333.pdf
working on: pdf_data\EU-2024-1689.pdf
OCR error on EU-2024-1689.pdf: Unable to get page count.
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table

working on: pdf_data\FL-S1680.pdf
working on: pdf_data\IL-H4705.pdf
working on: pdf_data\IL-H4836.pdf
working on: pdf_data\IL-H4844.pdf
working on: pdf_data\IN-AI_task_Force.pdf
working on: pdf_data\MD-818.pdf
working on: pdf_data\MEX.pdf
working on: pdf_data\NH_H1688.pdf
working on: pdf_data\NJ-S3357.pdf
working on: pdf_data\NY-A8129.pdf
working on: pdf_data\OJ_L_202401689_EN_TXT.pdf
working on: pdf_data\OR-H4153.pdf
working on: pdf_data\PA-H49.pdf
working on: pdf_data\PA-HR170.pdf
workin

Look at: https://github.com/lwdozal/Chat-with-your-Research-Articles-LLM-Retrieval-Augmented-Generation/blob/main/Intermediate%20RAG%20(prefilled).ipynb

To start creating embeddings for RAG (This might be an easier start for us, and might still hold some buzzword hype)

Tokenize sentences into paragraphs ~5 sentences into each chunk 

In [10]:
# legal_texts = pd.DataFrame(legal_texts)

with open('legal_pdf_data.json', 'r', encoding='utf-8') as f:
    pdf_data = json.load(f)

In [40]:
def split_legal_clauses(text, legal_clauses):
    # set-up splitter to Find the clauses in the text
    clause_splitter = re.compile(r'(?=({}))'.format('|'.join(legal_clauses)), re.IGNORECASE)

    # Split based on legal clause words
    clauses = clause_splitter.split(text)
    
    # Clean up chunks and stitch words to their content
    refined_clauses = []
    i = 0
    while i < len(clauses):
        if clause_splitter.match(clauses[i]):
            # Pair clause header with its content
            if i + 1 < len(clauses):
                refined_clauses.append((clauses[i] + ' ' + clauses[i+1]).strip())
                i += 2
            else:
                refined_clauses.append(clauses[i].strip())
                i += 1
        else:
            if clauses[i].strip():
                refined_clauses.append(clauses[i].strip())
            i += 1

    return refined_clauses

In [41]:

def split_text(text, legal_clauses, para_size=5, overlap=2):
    """
    Convert flat text into overlapping paragraphs of sentences.
    - para_size: number of sentences per paragraph
    - overlap: number of sentences to overlap between paragraphs
    """
    #Tokenize into sentences
    sentences = sent_tokenize(text)
    # split text based on clauses
    clauses = split_legal_clauses(text, legal_clauses)

    # split text to create overlapping paragraphs
    paragraphs = []
    i = 0
    while i < len(sentences):
        para = sentences[i:i+para_size]
        if para:
            paragraphs.append(' '.join(para))
        i += para_size - overlap  # Slide forward by (para_size - overlap)

    return paragraphs, sentences, clauses

In [42]:
# pdf_data

# Types of legal clause headers
LEGAL_CLAUSES = [
    r'WHEREAS\b',
    r'NOW, THEREFORE\b',
    r'BE IT RESOLVED\b',
    r'IN WITNESS WHEREOF\b',
    r'THIS AGREEMENT\b',
    r'FOR THE AVOIDANCE OF DOUBT\b',
    r'SUBJECT TO\b',
    r'NOTWITHSTANDING\b'
]

#chunk data into paragraphs and sentences for best Summarization and RAG
for doc in pdf_data:
    print("Working on:", doc["filename"])
    print("Number of words:", len(parsed.text))

    text = doc['text']
    text = " ".join(text)
    joined_text = text.replace("\n"," ")
    joined_text = joined_text.replace("\n\n"," ")
    parsed = nlp(text)
    # print(parsed.text)
    
    # split the text into sentences, identified clauses, and sentences
    paragraphs, sentences, clauses = split_text(parsed.text, LEGAL_CLAUSES, para_size=8, overlap=3)

    doc['paragraphs'] = paragraphs
    doc['sentences'] = sentences
    doc['sentences'] = clauses

with open('legal_pdf_tokenized.json', 'w', encoding='utf-8') as f:
    json.dump(pdf_data, f, ensure_ascii=False, indent=2)

ARG-1472-D-2023.pdf
Number of words 8162
CA-2885.pdf
Number of words 26023
CA-AB2013.pdf
Number of words 7471
CA-AJR6.pdf
Number of words 5339
CA-SB1047.pdf
Number of words 54031
CA-SB942.pdf
Number of words 11133
CO-CAIA.pdf
Number of words 42107
DE-H333.pdf
Number of words 6812
FL-S1680.pdf
Number of words 8586
IL-H4705.pdf
Number of words 4333
IL-H4836.pdf
Number of words 6078
IL-H4844.pdf
Number of words 3030138
IN-AI_task_Force.pdf
Number of words 18680
MD-818.pdf
Number of words 35879
MEX.pdf
Number of words 33008
NH_H1688.pdf
Number of words 7431
NJ-S3357.pdf
Number of words 7388
NY-A8129.pdf
Number of words 16263
OJ_L_202401689_EN_TXT.pdf
Number of words 597877
OR-H4153.pdf
Number of words 4712
PA-H49.pdf
Number of words 4490
PA-HR170.pdf
Number of words 10663
RI-S117.pdf
Number of words 8174
TN-H2325.pdf
Number of words 8748
UK.pdf
Number of words 212399
UT-S149.pdf
Number of words 35275
UT-SB0149.pdf
Number of words 39725
VA-H747.pdf
Number of words 20134
VA-S487.pdf
Number o